# 01 angr avoid

First we load the compiled binary executable:

In [54]:
import angr, sys

libname = "../build/arm64-v8a/01_angr_avoid"
project = angr.Project(libname, main_opts={ "base_addr": 0x00100000 })
print(project.arch, hex(project.entry))

<Arch AARCH64 (LE)> 0x100804


There is something special about libc for android ndk, angr can't handle some functions for us automatically, so we need to handle them manually.

In [55]:
project.hook_symbol("__strncpy_chk", angr.SIM_PROCEDURES['libc']['strncpy']())
project.hook_symbol("__strncpy_chk2", angr.SIM_PROCEDURES['libc']['strncpy']())


WARNING | 2022-05-24 13:56:18,102 | angr.project | Address is already hooked, during hook(0x200028, <SimProcedure strncpy>). Re-hooking.
WARNING | 2022-05-24 13:56:18,104 | angr.project | Address is already hooked, during hook(0x200030, <SimProcedure strncpy>). Re-hooking.


2097200

Now we first find the address of the main function:

In [56]:
main_func = project.loader.find_symbol("main")
print(hex(main_func.rebased_addr))

0x100974


We prepare the state for symbolic execution:

In [57]:
entry_state = project.factory.entry_state(addr=main_func.rebased_addr)
entry_state.options.add(angr.sim_options.ZERO_FILL_UNCONSTRAINED_MEMORY)
entry_state.options.add(angr.sim_options.ZERO_FILL_UNCONSTRAINED_REGISTERS)

We want to find the path that can reach the line of print "Good Job": (e.g.: 0x00100950)

In [58]:
print_good_address = int(input("good job address(hex): "), 16)

The binary is pretty large, to save you some time, everything you will need to look at is near the beginning of the address space.(e.g.: 0x0010090c)

In [59]:
will_not_succeed_address = int(input("will not succeed address(hex): "), 16)

Everything is ready, next we start the symbolic execution to find the path:

In [60]:
simulation = project.factory.simulation_manager(entry_state)
simulation.explore(find=print_good_address, avoid=will_not_succeed_address)
if simulation.found:
    print("Solution(password):", simulation.found[0].posix.dumps(sys.stdin.fileno()).decode())
else:
    print("Cound not find the solution")
    list(map(lambda s: s.callstack.dbg_repr(), simulation.deadended))

Solution(password): WEQPPXFR
